In [1]:
import numpy as np
import librosa
from scipy.io import wavfile
from IPython.display import Audio
import matplotlib.pyplot as plt
from scipy.signal import fftconvolve

/opt/anaconda3/envs/audiobasics/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/anaconda3/envs/audiobasics/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/anaconda3/envs/audiobasics/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [61]:
%config Completer.use_jedi = False


In [60]:
##copied
def get_time_smoothed_representation(
    spectral, samplerate, hop_length, time_constant_s=0.001
):
    t_frames = time_constant_s * samplerate / float(hop_length)
    # By default, this solves the equation for b:
    #   b**2  + (1 - b) / t_frames  - 2 = 0
    # which approximates the full-width half-max of the
    # squared frequency response of the IIR low-pass filt
    b = (np.sqrt(1 + 4 * t_frames ** 2) - 1) / (2 * t_frames ** 2)
    return filtfilt([b], [1, b - 1], spectral, axis=-1, padtype=None)

def sigmoid(x,shift,mult):
    
    return 1/(1+np.exp(-(x+shift)*mult))



In [ ]:
def remove_nonstationary_noise(audio):
    
    audio_stft = librosa.stft(audio,n_fft=n_fft,win_length=win_length,hop_length=hop_length)
    audio_stft_abs = np.abs(audio_stft)
    
    audio_time_smoothed = get_time_smoothed_representation(audio,sr,hop_length)
    
    audio_mask = (audio_stft_abs - audio_time_smoothed)/audio_time_smoothed
    audio_mask = sigmoid(audio_mask,shift,mult)
    
    smoothing_filter = np.outer(
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_freq + 1, endpoint=False),
                np.linspace(1, 0, n_grad_freq + 2),
            ]
        )[1:-1],
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_time + 1, endpoint=False),
                np.linspace(1, 0, n_grad_time + 2),
            ]
        )[1:-1],
    )
    
    audio_mask = fftconvolve(audio_mask,smoothing_filter,mode='same')
    audio_mask = audio_mask * prop_decrease + np.ones(audio_mask.shape) * (1-prop_decrease)
    
    denoised_audio_stft = audio_stft * audio_mask
    
    return librosa.istft(denoised_audio_stft,win_length=win_length,hop_length=hop_length)
    
    